*disclaimer* only to be used for this article and nothing else
[An Introduction to Asynchronous Programming in Python](https://medium.com/velotio-perspectives/an-introduction-to-asynchronous-programming-in-python-af0189a88bbb)

that being said Good luck man, you need it.

# An Introduction to Asynchronous Programming in Python

## Introduction
Asynchronous programming is a type of parallel programming in which a unit of work is allowed to run separately from the primary application thread. When the work is complete, it notifies the main thread about completion or failure of the worker thread. There are numerous benefits to using it, such as improved application performance and enhanced responsiveness.

![difference](https://miro.medium.com/max/540/1*t_oCyHBstMnF8WpZ67pKTg.jpeg)

For example, instead of waiting for an HTTP request to finish before continuing execution, with Python async coroutines you can submit the request and do other work that’s waiting in a queue while waiting for the HTTP request to finish.
Asynchronicity seems to be a big reason why Node.js so popular for server-side programming. Much of the code we write, especially in heavy IO applications like websites, depends on external resources. This could be anything from a remote database call to **POSTing** to a **REST** service. As soon as you ask for any of these resources, your code is waiting around with nothing to do. With asynchronous programming, you allow your code to handle other tasks while waiting for these other resources to respond.




### 1. Multiple Processes
The most obvious way is to use multiple processes. From the terminal, you can start your script two, three, four…ten times and then all the scripts are going to run independently or at the same time. The operating system that’s underneath will take care of sharing your **CPU** resources among all those instances. Alternately you can use the **multiprocessing library** which supports *spawning processes* as shown in the example below.


In [1]:
# Code Deomonstrating Multiple Processes

from multiprocessing import Process


def print_func(continent='Asia'):
    print('The name of continent is : ', continent)

if __name__ == "__main__":  # confirms that the code is under main function
    names = ['America', 'Europe', 'Africa']
    procs = []
    proc = Process(target=print_func)  # instantiating without any argument
    procs.append(proc)
    proc.start()

    # instantiating process with arguments
    for name in names:
        # print(name)
        proc = Process(target=print_func, args=(name,))
        procs.append(proc)
        proc.start()

    # complete the processes
    for proc in procs:
        proc.join()

The name of continent is :  AsiaThe name of continent is : 
 AmericaThe name of continent is :  
EuropeThe name of continent is : 
 Africa


## 2. Multiple Threads
The next way to run multiple things at once is to use **threads**. A thread is a line of execution, pretty much like a process, *but you can have multiple threads in the context of one process and they all share access to common resources*. But because of this, it’s difficult to write a threading code. And again, the operating system is doing all the heavy lifting on sharing the CPU, but the global interpreter lock (GIL) allows only one thread to run Python code at a given time even when you have multiple threads running code. So, In CPython(CPython is the reference implementation of the Python programming language.), the GIL prevents multi-core concurrency. Basically, you’re running in a single core even though you may have two or four or more.

In [2]:
# Code Demonstration of Multiple Threading

import threading

def print_cube(num):
    """
    function to print cube of given num
    """
    print("Cube: {}".format(num * num * num))
 
def print_square(num):
    """
    function to print square of given num
    """
    print("Square: {}".format(num * num))
 
if __name__ == "__main__":
    # creating thread
    t1 = threading.Thread(target=print_square, args=(10,))
    t2 = threading.Thread(target=print_cube, args=(10,))
 
    # starting thread 1
    t1.start()
    # starting thread 2
    t2.start()
 
    # wait until thread 1 is completely executed
    t1.join()
    # wait until thread 2 is completely executed
    t2.join()
 
    # both threads completely executed
    print("Done!")

Square: 100
Cube: 1000
Done!


## 3. Coroutines using yield:
Coroutines are generalization of **subroutines**. They are used for cooperative multitasking where a process voluntarily yield (give away) control periodically or when idle in order to enable multiple applications to be run simultaneously. Coroutines are similar to generators but with few extra methods and slight change in how we use yield statement. Generators produce data for iteration while coroutines can also consume data.

Subroutines - `mini programs` within a large program.
        Breaking down or decomposing a complex programming task into smaller sub-tasks and writing each of these as subroutines, makes the problem easier to solve


### yield
The yield statement suspends function’s execution and sends a value back to the caller, but *retains enough state* to enable function to resume where it is left off. When resumed, the function continues execution immediately *after the last yield run*. This allows its code to produce a series of values over time, rather than computing them at once and sending them back like a list.

sends a series of values over time > sending back a list

Technically speaking, a Python iterator object must implement two special methods, `__iter__()` and `__next__()`, collectively called the **iterator protocol**.

`corou.__next__()`
`next(corou)`

both syntax is correct



In [21]:
def print_name(prefix):
    print("Searching prefix:{}".format(prefix))
    try : 
        while True:
                # yield used to create coroutine (yield is keyword)
                name = (yield)
                if prefix in name:
                    print(name)
    except GeneratorExit:
            print("Closing coroutine!!")
 
corou = print_name("Dear")
# this skips a value both syntax corrent 
# corou.__next__()
next(corou)

# methods of class generator.
# becomes generator when yield is used in function body.
corou.send("James")
corou.send("Dear James")
# print(type(corou), dir(corou), corou, sep='\n')
corou.close()

Searching prefix:Dear
Dear James
Closing coroutine!!


In [10]:
# Understanding how Yield works

# yield is a really wild return statement

def simpleGeneratorFun():
    yield 1
    yield 2
    yield 3
    yield 'can send anything over'
  
# Driver code to check above generator function
for value in simpleGeneratorFun(): 
    print(value)
    # print('\n')


1
2
3
can send anything over


**Return** sends a specified value back to its caller whereas **Yield** can produce a sequence of values. We should use yield when we want to iterate over a sequence, but don’t want to store the entire sequence in memory.

Yield are used in Python **generators**. A generator function is defined like a normal function, but whenever it needs to generate a value, it does so with the yield keyword rather than return. **IMP** *If the body of a def contains yield, the function automatically becomes a generator function.*

if it contains yield it becomes a generator function.


## 4. Asynchronous Programming
The fourth way is an asynchronous programming, where the **OS is not participating**. As far as OS is concerned you’re going to have one process and there’s going to be a single thread within that process, but you’ll be able to do multiple things at once. So, what’s the trick?
*The answer is asyncio*\
asyncio is the new concurrency module introduced in Python 3.4. It is designed to use coroutines and futures to simplify asynchronous code and make it almost as *readable as synchronous code* as there are no callbacks.


**asyncio** uses different constructs: *event loops, coroutines and futures.*
More like Js experience then


* An event loop manages and distributes the execution of different tasks. It registers them and handles distributing the flow of control between them.


* Coroutines (covered above) are special functions that work similarly to Python generators, on await they release the flow of control back to the event loop. A coroutine needs to be scheduled to run on the event loop, once scheduled coroutines are wrapped in Tasks which is a type of Future.


* Futures represent the result of a task that may or may not have been executed. This result may be an exception.\

Using *Asyncio*, you can structure your code so `subtasks are defined as coroutines` and allows you to schedule them as you please, including simultaneously. Coroutines contain yield points where we define possible points where a context switch can happen if other tasks are pending, but will not if no other task is pending.
A context switch in asyncio represents the event loop yielding the flow of control from one coroutine to the next.
In the example, we run 3 async tasks that query Reddit separately, extract and print the JSON. We leverage aiohttp which is a http client library ensuring even the HTTP request runs asynchronously.

In [5]:
# defining custom handlers to be executed when a signal is received
import signal
import asyncio

# Asynchronous HTTP Client/Server for asyncio and Python.
import sys
import aiohttp
import json

# created event loop
loop = asyncio.get_event_loop()
# Client session is the recommended interface for making HTTP requests.
# initiated Session
client = aiohttp.ClientSession(loop=loop)

async def get_json(client, url):  
    async with client.get(url) as response:
        assert response.status == 200
        return await response.read()


# https://www.reddit.com/r/%7Bsubreddit%7D/top.json?sort=top&t=day&limit=5
# returns json
async def get_reddit_top(subreddit, client):  
    data1 = await get_json(client, 'https://www.reddit.com/r/' + subreddit + '/top.json?sort=top&t=day&limit=5')

    j = json.loads(data1.decode('utf-8'))
    for i in j['data']['children']:
        score = i['data']['score']
        title = i['data']['title']
        link = i['data']['url']
        print(str(score) + ': ' + title + ' (' + link + ')')
        print('\n \n \n ')

    print('DONE:', subreddit + '\n')

def signal_handler(signal, frame):  
    loop.stop()
    client.close()
    sys.exit(0)

signal.signal(signal.SIGINT, signal_handler)

asyncio.ensure_future(get_reddit_top('python', client))  
asyncio.ensure_future(get_reddit_top('programming', client))  
asyncio.ensure_future(get_reddit_top('compsci', client))  
# loop.run_forever()

<Task pending name='Task-47' coro=<get_reddit_top() running at <ipython-input-5-c6253fecffdb>:24>>

Task exception was never retrieved
future: <Task finished name='Task-45' coro=<get_reddit_top() done, defined at <ipython-input-5-c6253fecffdb>:24> exception=AssertionError()>
Traceback (most recent call last):
  File "<ipython-input-5-c6253fecffdb>", line 25, in get_reddit_top
    data1 = await get_json(client, 'https://www.reddit.com/r/' + subreddit + '/top.json?sort=top&t=day&limit=5')
  File "<ipython-input-5-c6253fecffdb>", line 18, in get_json
    assert response.status == 200
AssertionError
Task exception was never retrieved
future: <Task finished name='Task-46' coro=<get_reddit_top() done, defined at <ipython-input-5-c6253fecffdb>:24> exception=AssertionError()>
Traceback (most recent call last):
  File "<ipython-input-5-c6253fecffdb>", line 25, in get_reddit_top
    data1 = await get_json(client, 'https://www.reddit.com/r/' + subreddit + '/top.json?sort=top&t=day&limit=5')
  File "<ipython-input-5-c6253fecffdb>", line 18, in get_json
    assert response.status == 200
Assertion

[asyncio // stackoverflow difference between python3 and jupyter](https://stackoverflow.com/questions/55409641/asyncio-run-cannot-be-called-from-a-running-event-loop)

`asyncio.run(main())` - Python (≥ 3.7)
`await main()` - Jupyter / iPython

somehow jupyter is already running an event loop


Real Python // asyncio
**Parallelism** consists of performing multiple operations at the same time. **Multiprocessing** is a means to effect parallelism, and it entails spreading tasks over a computer’s central processing units (CPUs, or cores). Multiprocessing is well-suited for CPU-bound tasks: tightly bound for loops and mathematical computations usually fall into this category.\

**Concurrency** is a slightly broader term than parallelism. It suggests that multiple tasks have the ability to run in an overlapping manner. *(There’s a saying that concurrency does not imply parallelism.)*

GIL - Global Interpreter Lock
The Python Global Interpreter Lock or GIL, in simple words, is a mutex (or a lock) that allows only one thread to hold the control of the Python interpreter.

This means that only one thread can be in a state of execution at any point in time. The impact of the GIL isn’t visible to developers who execute single-threaded programs, but it can be a performance bottleneck in CPU-bound and multi-threaded code.

Since the GIL allows only one thread to execute at a time even in a multi-threaded architecture with more than one CPU core, the GIL has gained a reputation as an “infamous” feature of Python.

[Real python // GIL](https://realpython.com/python-gil/)
GIL is a necessary Devil?

This is how we're circumventing GIL
> **Multi-processing** vs **multi-threading**: The most popular way is to use a multi-processing approach where you use multiple processes instead of threads. Each Python process gets its own *Python interpreter* and memory space so the GIL won’t be a problem. Python has a **multiprocessing** module which lets us create processes easily like this:

below are code examples to help illustrate the differences.
Different threads don't help due to GIL, 
however, Multiprocessing helps.
there is a little bit of overhead cost, but overall its pog.


In [19]:
# MULTIPROCESSING MODULE
from multiprocessing import Pool
import time

COUNT = 50000000
def countdown(n):
    while n>0:
        n -= 1

if __name__ == '__main__':
    pool = Pool(processes=2)
    start = time.time()
    r1 = pool.apply_async(countdown, [COUNT//2])
    r2 = pool.apply_async(countdown, [COUNT//2])
    pool.close()
    pool.join()
    end = time.time()
    print('Time taken in seconds -', end - start)

/usr/lib/python3.8/multiprocessing/context.py:118: RuntimeWarning: coroutine 'main' was never awaited
  from .pool import Pool


Time taken in seconds - 2.554898262023926


In [20]:
# single_threaded.py
import time
from threading import Thread

COUNT = 50000000

def countdown(n):
    while n>0:
        n -= 1

start = time.time()
countdown(COUNT)
end = time.time()

print('Time taken in seconds -', end - start)

Time taken in seconds - 3.671034812927246


In [21]:
# multi_threaded.py
import time
from threading import Thread

COUNT = 50000000

def countdown(n):
    while n>0:
        n -= 1

t1 = Thread(target=countdown, args=(COUNT//2,))
t2 = Thread(target=countdown, args=(COUNT//2,))

start = time.time()
t1.start()
t2.start()
t1.join()
t2.join()
end = time.time()

print('Time taken in seconds -', end - start)

Time taken in seconds - 4.623546123504639


In [18]:
import asyncio

async def main():
    print('hello')
    await asyncio.sleep(1)
    print('bye')

await (main())


hello
bye
